# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,9.91,71,75,0.00,SJ,1691389211
1,1,college,64.8569,-147.8028,22.67,64,75,2.06,US,1691389011
2,2,yuzhno-kurilsk,44.0306,145.8556,15.39,93,100,3.34,RU,1691389212
3,3,avarua,-21.2078,-159.7750,22.03,88,75,2.06,CK,1691389081
4,4,jamestown,42.0970,-79.2353,22.48,100,100,2.57,US,1691389050


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
display(map_plot_1)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
#       A max temperature lower than 27 degrees but higher than 21
#       Wind speed less than 4.5 m/s
#       Zero cloudiness
ideal_weather_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5) & \
                                    (city_data_df["Max Temp"] < 27.0) & (city_data_df["Max Temp"] > 21.0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
print(ideal_weather_df.count())
ideal_weather_df.head()

City_ID       24
City          24
Lat           24
Lng           24
Max Temp      24
Humidity      24
Cloudiness    24
Wind Speed    24
Country       24
Date          24
dtype: int64


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,talnakh,69.4865,88.3972,24.56,33,0,1.27,RU,1691389223
96,96,hawaiian paradise park,19.5933,-154.9731,23.91,94,0,2.06,US,1691389258
108,108,khabary,53.6259,79.5345,24.91,31,0,4.14,RU,1691389265
128,128,sao vicente,-23.9631,-46.3919,25.15,26,0,2.06,BR,1691389273
137,137,kharp,66.8014,65.8081,25.62,54,0,2.12,RU,1691389278


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(len(hotel_df))
hotel_df.head()

24


C:\Users\andre\AppData\Local\Temp\ipykernel_25392\2374050864.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
21,talnakh,RU,69.4865,88.3972,33,
96,hawaiian paradise park,US,19.5933,-154.9731,94,
108,khabary,RU,53.6259,79.5345,31,
128,sao vicente,BR,-23.9631,-46.3919,26,
137,kharp,RU,66.8014,65.8081,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
talnakh - nearest hotel: Талнах
hawaiian paradise park - nearest hotel: No hotel found
khabary - nearest hotel: Гостиница "Элегия" с. Хабары
sao vicente - nearest hotel: Pousada Vitória
kharp - nearest hotel: Собь
ierapetra - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
castaic - nearest hotel: No hotel found
santander jimenez - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
nantucket - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
waddan - nearest hotel: No hotel found
konya - nearest hotel: No hotel found
san ignacio - nearest hotel: Rainforest Haven Inn
alaghsas - nearest hotel: Hôtel Zagado
mount pleasant - nearest hotel: Old Village Post House
arroyo de la luz - nearest hotel: Hotel Rural Villa Matilde
nizhniy lomov - nearest hotel: Нижний Ломов
vanino - nearest hotel: Пять Звёзд
portland - nearest hotel: the Hoxton
franklin - nearest hotel: Court

,City,Country,Lat,Lng,Humidity,Hotel Name
21,talnakh,RU,69.4865,88.3972,33,Талнах
96,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
108,khabary,RU,53.6259,79.5345,31,"Гостиница ""Элегия"" с. Хабары"
128,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
137,kharp,RU,66.8014,65.8081,54,Собь
156,ierapetra,GR,35.0100,25.7353,73,No hotel found
190,tsiombe,MG,-25.3000,45.4833,61,No hotel found
197,castaic,US,34.4889,-118.6229,48,No hotel found
199,santander jimenez,MX,24.2167,-98.4667,83,No hotel found
200,hilo,US,19.7297,-155.0900,86,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
display(map_plot_2)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)